In [1]:
import pyfair

import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline
import pandas as pd
import numpy as np
import os

In [1]:
import pyfair

ModuleNotFoundError: No module named 'pyfair.model.model_dependency'

In [ ]:
model1 = pyfair.FairModel(name="HA1", n_simulations=10_000)
model1.input_data('Loss Event Frequency', low=20, mode=100, high=900)
model1.input_data('Probable Loss Magnitude', low=3_000_000, mode=3_500_000, high=5_000_000)
model1.calculate_all()

model2 = pyfair.FairModel(name="HA2", n_simulations=10_000)
model2.input_data('Loss Event Frequency', low=.5, mode=.7, high=.9)
model2.input_data('Probable Loss Magnitude', low=2_000_000_000, mode=3_000_000_000, high=5_000_000_000)
model2.calculate_all()

model3 = pyfair.FairModel(name="HA3", n_simulations=10_000)
model3.input_data('Loss Event Frequency', low=10, mode=15, high=16)
model3.input_data('Probable Loss Magnitude', low=500_000_000, mode=500_900_000, high=600_000_000)
model3.calculate_all()

#fr = FairReport(model1)
#fr.to_html('C:/Users/theon/Desktop/report.html')
results = model1.export_results()
node_statuses = model1.get_node_statuses()

mm = pyfair.FairMetaModel(name='My Meta Model!', models=[model1, model3])
mm.calculate_all()

fsr = pyfair.FairSimpleReport([model2, mm])
fsr.to_html('output.html')

In [ ]:
mm = pyfair.FairMetaModel(name='My Meta Model!', models=[model1, model2])
mm.calculate_all()

fsr = pyfair.FairSimpleReport([model2, mm])
fsr.to_html('output.html')

# Components

<section header and metadata>

<section overview>

* Summary General
    * Summary Table (mean, stdev, low, high, n)
    * hist
    * Exceedence curves
* Summary Comparison
    * Summary Table (mean, stdev, low, high, n)
    * Summary Table 2
    * Combined Hist
    * Exceedence curves
    
<section parameters>

* Model Detail
    * Parameter table with distributions
* Metamodel Detail
    * Risk table with stacked area
    
<section> and source code
* JSON

# Reports
* Compare
* Single
* Metamodel Drilldown

In [ ]:

pd.DataFrame({
    model1.get_name(): model1.export_results()['Risk'],
    model2.get_name(): model2.export_results()['Risk'],    
}).agg([np.mean, np.std, np.min, np.max])



In [ ]:
from pyfair.report.distribution import FairDistributionCurve

fdc = FairDistributionCurve([model1, model2, model3])
#fig, ax = fdc.generate_image()

fig, ax = fdc.generate_image()
fdc.generate_icon('HA1', 'Risk')

In [ ]:
from pyfair.report.exceedence import FairExceedenceCurves

fpc = FairExceedenceCurves([model1, model2, model3])
fig1, ax1 = fpc.generate_image()

In [5]:
import numpy as np

'func1'

'func1'

In [8]:
import scipy.stats

import numpy as np
import pandas as pd

#from ..utility.fair_exception import FairException
#from ..utility.beta_pert import FairBetaPert
from pyfair.utility import FairException
from pyfair.utility import FairBetaPert



class FairDataInput(object):
    '''Data entry and validation.
    
    TODO: confirm accuracy of these function mappings.
    
    '''
    
    def __init__(self):
        # These targets must be less than or equal to one
        self._le_1_targets = ['Contact', 'Action', 'Vulnerability', 'Control Strength']
        self._le_1_keywords = ['constant', 'high', 'mode', 'low', 'mean', 'p']
        # Parameter map associates parameters with functions
        self._parameter_map = {
            'constant': self._gen_constant,
            'high'    : self._gen_pert,
            'mode'    : self._gen_pert,
            'low'     : self._gen_pert,
            'gamma'   : self._gen_pert,
            'mean'    : self._gen_normal,
            'stdev'   : self._gen_normal,
            'p'       : self._gen_bernoulli,
        }
        # Vulnerability is bernoulli only.
        self._bernoulli_targets = ['Vulnerability']
        # List of keywords with function keys
        self._required_keywords = {
            self._gen_constant : ['constant'],
            self._gen_pert     : ['low', 'mode', 'high'],
            self._gen_normal   : ['mean', 'stdev'],
            self._gen_bernoulli: ['p']
        }  
        # Storage of inputs
        self._supplied_values = {}

    def get_supplied_values(self):
        '''Fetch params'''
        return self._supplied_values
    
    def _check_le_1(self, target, **kwargs):
        '''Checks if certain keyword arguments are less than 1'''
        # For every keyword argument
        for key, value in kwargs.items():
            # If key is in specified list
            if key in self._le_1_keywords:
                # Check if value is less than or equal to 1
                if 0.0 <= value <= 1.0:
                    pass
                # If not, raise error
                else:
                    raise FairException('"{}" must have "{}" value between zero and one.'.format(target, key))

    def _check_parameters(self, target_function, **kwargs):
        '''Look up keywords based on function type'''
        required_keywords = self._required_keywords[target_function]
        for required_keyword in required_keywords:
            if required_keyword in kwargs.keys():
                pass
            else:
                raise FairException('"{}" is missing "{}".'.format(str(target_function), required_keyword))

    def generate(self, target, count, **kwargs):
        '''Function for dispatching a generation request'''
        # If destined for a le_1_target, check validity.
        if target in self._le_1_targets:
            self._check_le_1(target, **kwargs)
        # If target is bernoulli, shunt into that function.
        if target in self._bernoulli_targets:
            results = self._gen_bernoulli(count, **kwargs)
        else:
            # Otherwise figure out what function
            func = self._determine_func(**kwargs)
            # Check to make sure sufficient parameters exist
            self._check_parameters(func, **kwargs)
            # Run the function
            results = func(count, **kwargs)
        # Record params
        self._supplied_values[target] = {**kwargs}
        return results
            
    def _determine_func(self, **kwargs):
        '''This function takes keywords and determines function'''
        # Check whether keys are recognized
        for key in kwargs.keys():
            if not key in self._parameter_map.keys():
                raise FairException('"{}"" is not a recognized keyword'.format(key))
        # Check whether all keys go to same function via set comprension
        functions = list(set([
            self._parameter_map[key]
            for key
            in kwargs.keys()
        ]))
        if len(functions) > 1:
            raise FairException('"{}" mixes incompatible keywords.'.format(str(kwargs.keys())))
        else:
            function = functions[0]
            return function

    def _gen_bernoulli(self, count, **kwargs):
        # No check required as 0 to 1 is already esablished
        bernoulli = scipy.stats.bernoulli(**kwargs)
        rvs = bernoulli.rvs(count)
        return rvs
    
    def _gen_constant(self, count, **kwargs):
        return np.full(count, kwargs['constant'])
    
    def _gen_normal(self, count, **kwargs):
        normal = scipy.stats.norm(loc=kwargs['mean'], scale=kwargs['stdev'])
        rvs = normal.rvs(count)
        # Clip out of range values
        clipped_rvs = np.clip(rvs, 0.0, 1.0)
        return clipped_rvs
    
    def _gen_pert(self, count, **kwargs):
        self._check_pert(**kwargs)
        pert = FairBetaPert(**kwargs)
        rvs = pert.random_variates(count)
        return rvs
    
    def _check_pert(self, **kwargs):
        conditions = {
            'mode >= low'  : kwargs['mode'] >= kwargs['low'],
            'high >= mode' : kwargs['high'] >= kwargs['mode'],
        }
        for condition_name, condition_value in conditions.items():
            if condition_value == False:
                err = 'Param "{}" fails PERT requirement "{}".'.format(kwargs, condition_name)
                raise FairException(err)


In [15]:
import matplotlib
%matplotlib inline
import matplotlib.pyplot as plt

fdi = FairDataInput()
data = fdi.generate('Control Strength', count=10_000, low=.2, high=1.5, mode=.4)

plt.hist(data)

FairException: Control Strength must have high value between zero and one.